<a href="https://colab.research.google.com/github/alessandronascimento/BioMolComp/blob/main/P10/TFBinding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sítio de Interação de Fator de Transcrição


Fatores de transcrição são tipicamente proteínas que reconhecem sequências específicas, e de tamanho específico, do DNA e regulam a transcrição de genes nas adjacências do sítio de interação.

Os sítio de interação tipicamente contem ~10 bp com uma assinatura de sequência mais ou menos conhecida (e.g.: AGGTCA).

## Fator de Transcrição JUND

As sequências de DNA que são reconhecidas pelo fator de transcrição JUND foram mapeadas exaustivamente no genoma humano. Neste experimento, vamos usar apenas os dados do cromossomo 22 (um dos menores), e que conta com 50 milhões de bases. Neste experimento, elas foram divididas em segmentos de 101 pares de bases e foram rotuladas para indicar a ligação ou não do JUND a este segmento.

Novamente, vamos usar o *one-hot encoded* para representar as sequências: vetores de quatro elementos contendo um 1 e três 0, para representar A,G,C ou T.

Novamente vamos usar uma rede CNN, mas desta vez uma rede 1D, já que o dado é também 1D.

## Inicialmente, vamos instalar as bibliotecas necessárias:

In [ ]:
# Train a model to predict binding sites for the transcription factor JUND.
!pip3 install deepchem
!apt-get install subversion
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
%matplotlib inline
import matplotlib.pyplot as plt
!nvidia-smi

## Carregando os dados

Os dados que usaremos serão obtidos do GitHub com os comandos da célula abaixo.

In [3]:
# Get the data and load into deepchem

!svn export https://github.com/deepchem/DeepLearningLifeSciences/trunk/Chapter06/train_dataset
!svn export https://github.com/deepchem/DeepLearningLifeSciences/trunk/Chapter06/valid_dataset

# Load the data.

train = dc.data.DiskDataset('train_dataset')
valid = dc.data.DiskDataset('valid_dataset')

svn: E155000: Destination directory exists; please remove the directory or use --force to overwrite
svn: E155000: 'train_dataset' already exists
A    valid_dataset
A    valid_dataset/metadata.csv.gzip
A    valid_dataset/shard-0-X.joblib
A    valid_dataset/shard-0-ids.joblib
A    valid_dataset/shard-0-w.joblib
A    valid_dataset/shard-0-y.joblib
A    valid_dataset/tasks.json
Exported revision 55.


## Construção do Modelo

Na célula abaixo, construiremos o nosso modelo. A lógica é muito similar àquela usada no conjunto de dados MNIST. Aqui o modelo de convolução unidimensional (*Conv1D*) é empregado recebendo como input vetores de características de 101x4 elementos. Três camadas convolutivas são construídas com parâmetros idênticos. A largura do *kernel* convolutivo é de 10 e cada camada inclui 15 filtros (outputs).

O *dropout* é de 0.5, indicando que após cada camada, 50% dos valores de output são randomicamente setados para 0. Isto ajuda a prevenir o overfitting.

A camada densa de saída passa por uma função sigmoidal para que o resultado seja de valores entre 0 e 1, indicando a probabilidade de cada sequência conter um sítio de interação.

In [4]:
# Arquitetura do modelo

features = tf.keras.Input(shape=(101, 4))
prev = features
for i in range(3):
    prev = layers.Conv1D(filters=15, kernel_size=10, activation=tf.nn.relu, padding='same')(prev)
    prev = layers.Dropout(rate=0.5)(prev)
logits = layers.Dense(units=1)(layers.Flatten()(prev))
output = layers.Activation(tf.math.sigmoid)(logits)
keras_model = tf.keras.Model(inputs=features, outputs=[output, logits])
model = dc.models.KerasModel(
    keras_model,
    loss=dc.models.losses.SigmoidCrossEntropy(),
    output_types=['prediction', 'loss'],
    batch_size=1000,
    model_dir='tf')

## Treinamento e Métrica

Novamente a métrica será a ROC AUC. A cda 10 épocas, o modelo é avaliado nos conjuntos de treinamento e teste.

In [ ]:
# Treinamento do modelo e avaliação

metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

xs=[]
ytrain=[]
yvalid=[]

for i in range(20):
    model.fit(train, nb_epoch=10)
    print("Iteration %d:" % (i+1))
    m = model.evaluate(train, [metric])
    auc = m['roc_auc_score']
    print("ROC AUC [train]: %7.3f" % auc)
    ytrain.append(auc)
    m = model.evaluate(valid, [metric])
    auc = m['roc_auc_score']
    print("ROC AUC [valid]: %7.3f" % auc)
    yvalid.append(auc)
    xs.append(i+1)
    print()

## Gráfico: Evolução do Treinamento da Rede

In [ ]:
# Representação Gráfica da Evolução do Treinamento

plt.style.use('seaborn-whitegrid')
plt.ylabel('ROC AUC')
plt.xlabel('Iteration')
plt.plot(xs, ytrain, '-ok', label='train')
plt.plot(xs, yvalid, '-or', label='valid')